<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/ACL-cleanup/EEC_with_CN(ElMo%2C_BERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
t = np.transpose
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
##Install flair
!pip install -q flair allennlp

    100% |████████████████████████████████| 81kB 5.0MB/s 
    100% |████████████████████████████████| 5.6MB 6.3MB/s 
    100% |████████████████████████████████| 798kB 22.5MB/s 
    100% |████████████████████████████████| 122kB 28.8MB/s 
    100% |████████████████████████████████| 245kB 27.9MB/s 
    100% |████████████████████████████████| 552kB 24.2MB/s 
    100% |████████████████████████████████| 1.5MB 16.2MB/s 
    100% |████████████████████████████████| 51kB 20.5MB/s 
    100% |████████████████████████████████| 51kB 20.3MB/s 
    100% |████████████████████████████████| 512kB 24.3MB/s 
    100% |████████████████████████████████| 4.5MB 8.1MB/s 
    100% |████████████████████████████████| 133kB 31.3MB/s 
    100% |████████████████████████████████| 1.0MB 17.9MB/s 
    100% |████████████████████████████████| 61kB 22.5MB/s 
    100% |████████████████████████████████| 2.3MB 13.3MB/s 
    100% |████████████████████████████████| 143kB 29.8MB/s 
    100% |████████████████████████████████| 51k

In [3]:
##Using flair to load emlo small embeddinngs, which is in 768 dimensions
from flair.embeddings import BertEmbeddings, ELMoEmbeddings
from flair.data import Sentence


# init embedding
elmo_embedding = ELMoEmbeddings('small') #BertEmbeddings('bert-large-uncased')
bert_embedding = BertEmbeddings('bert-large-uncased')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


100%|██████████| 1248501532/1248501532 [00:35<00:00, 35208210.58B/s]


In [4]:
##Load EEC Data
!wget http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
!unzip Equity-Evaluation-Corpus.zip
##Load Packages
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

##Read EEC
EEC = pd.read_csv('/content/Equity-Evaluation-Corpus.csv', header=0)


--2019-03-15 02:18:29--  http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
Resolving www.saifmohammad.com (www.saifmohammad.com)... 192.185.17.122
Connecting to www.saifmohammad.com (www.saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102568 (100K) [application/zip]
Saving to: ‘Equity-Evaluation-Corpus.zip’

Equity-Evaluation-C 100%[===================>] 100.16K   573KB/s    in 0.2s    

2019-03-15 02:18:30 (573 KB/s) - ‘Equity-Evaluation-Corpus.zip’ saved [102568/102568]

Archive:  Equity-Evaluation-Corpus.zip
  inflating: Equity-Evaluation-Corpus.csv  
  inflating: README.txt              
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Load BERT

In [5]:
!gdown https://drive.google.com/uc?id=1JyRVLZb7IdiMGGwPfXuA1f-CmCvxHDXx

Downloading...
From: https://drive.google.com/uc?id=1JyRVLZb7IdiMGGwPfXuA1f-CmCvxHDXx
To: /content/bert_conceptors.tar.bz2
656MB [00:04, 134MB/s]


In [6]:
!tar xvjf /content/bert_conceptors.tar.bz2

bert/
bert/bert_WEFAT_1_Male_attributes.pkl
bert/big_bert_gender_list_propernouns.pkl
bert/bert_W_5_African_American_names.pkl
bert/bert_W_2_Weapons.pkl
bert/bert_W_1_Pleasant.pkl
bert/bert_W_5_European_American_names.pkl
bert/bert_W_5_Unpleasant.pkl
bert/bert_W_6_Male_names.pkl
bert/big_bert_gender_list_all.pkl
bert/bert_W_10_Pleasant.pkl
bert/bert_W_1_Insects.pkl
bert/bert_W_5_Unused_full_list_European_American_names.pkl
bert/bert_W_10_Young_peoples_names.pkl
bert/bert_W_6_Female_names.pkl
bert/bert_W_9_Physical_disease.pkl
bert/bert_W_7_Female_terms.pkl
bert/big_bert_gender_list_pronouns.pkl
bert/bert_W_3_Unused_full_list_European_American_names.pkl
bert/bert_W_4_Unused_full_list_African_American_names.pkl
bert/bert_W_8_Male_terms.pkl
bert/bert_WEFAT_2_Androgynous_Names.pkl
bert/bert_WEFAT_1_Careers.pkl
bert/bert_W_9_Temporary.pkl
bert/bert_W_2_Unpleasant.pkl
bert/bert_WEFAT_1_Female_attributes.pkl
bert/bert_W_2_Instruments.pkl
bert/bert_W_7_Male_terms.pkl
bert/bert_W_3_Unpleasant.p

# Load elmo race list embeddings

In [7]:
!gdown https://drive.google.com/uc?id=1WJBoEHOBkG4EW1MkB0y1QpggtV_J0i9w

Downloading...
From: https://drive.google.com/uc?id=1WJBoEHOBkG4EW1MkB0y1QpggtV_J0i9w
To: /content/elmo_race_list.pkl
4.88MB [00:00, 43.0MB/s]


# Load pre-calculated CN processed ELMo and BERT embeddings

In [0]:
import pickle

In [0]:
x = pickle.load(open('/content/elmo_race_list.pkl','rb'))
#x.keys()
#dict_keys(['GnegC', 'type_embedings', 'words'])
negC = x['GnegC']
negC_elmo = np.squeeze(np.asarray(negC))


In [0]:
x = pickle.load(open('/content/bert/big_bert_race_list.pkl','rb'))
#x.keys()
#dict_keys(['GnegC', 'type_embedings', 'words'])
negC = x['GnegC']
negC_bert = np.squeeze(np.asarray(negC))

# Load extra BERT for sentiment word embeddings

In [11]:
!gdown https://drive.google.com/uc?id=1dBiXlxSxfKq0-OhFQyL_CCij8pmThQZI

Downloading...
From: https://drive.google.com/uc?id=1dBiXlxSxfKq0-OhFQyL_CCij8pmThQZI
To: /content/bert_EEC_sentiment.pkl
100% 65.8k/65.8k [00:00<00:00, 25.6MB/s]


In [0]:
senn = pickle.load(open('/content/bert_EEC_sentiment.pkl','rb'))


In [13]:
senn['words']

['fear', 'sadness', 'joy', 'anger']

# Racial Debiasing

# Load brown corpus to get sentiment embeddings based on ELMo

In [0]:
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

In [15]:
## Get brown_emb 
!gdown https://drive.google.com/uc?id=17TK2h3cz7amgm2mCY4QCYy1yh23ZFWDU
!pip3 install -q pymagnitude
from pymagnitude import *
from pymagnitude import MagnitudeUtils
import numpy as np
import pickle
with open('/content/elmo_embeddings_emma_brown.pkl', 'rb') as f:
    elmo_data = pickle.load(f)
elmo_data = elmo_data['brown_embs']

## Get brown corpus
import nltk
nltk.download('brown')
from nltk.corpus import brown
brown_corpus = brown.sents()

Downloading...
From: https://drive.google.com/uc?id=17TK2h3cz7amgm2mCY4QCYy1yh23ZFWDU
To: /content/elmo_embeddings_emma_brown.pkl
4.78GB [01:57, 40.6MB/s]
    100% |████████████████████████████████| 5.4MB 8.0MB/s 
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [0]:
def pick_embeddings(corpus, sent_embs, word_list):
    X = []
    #labels = []
    #sents = []
    for i, s in enumerate(corpus):
       # print(s)
        for j, w in enumerate(s):
            if w in word_list:
                X.append(sent_embs[i][j])
                #labels.append(w)
                #sents.append(s)
    return X


# Run experiment-ELMO


In [0]:
def run_raw(sen_emb, sent1, sent2):
  
  sentence1 = Sentence(sent1)
  sentence2 = Sentence(sent2)
  x = embedding.embed(sentence1) # get sentence embedding of sentence 1
  se = torch.stack([token.embedding for token in sentence1])
  sent1_emb = se.numpy()
  x = embedding.embed(sentence2) # get sentence embedding of sentence 2
  se = torch.stack([token.embedding for token in sentence2])
  sent2_emb = se.numpy()
  sent1_emb = np.average(sent1_emb, axis = 0) #average sentence1 embeddings
  sent2_emb = np.average(sent2_emb, axis = 0) #average sentence2 embeddings
  proj1 = (sent1_emb/norm(sent1_emb)).dot(sen_emb)/norm(sen_emb) #cosine similarity
  proj2 = (sent2_emb/norm(sent2_emb)).dot(sen_emb)/norm(sen_emb)
  #print(sent2_emb)
  #print(proj1, proj2)
  return proj1, proj2

In [0]:
def run_conceptor(sen_emb, sent1, sent2):
  sentence1 = Sentence(sent1) 
  sentence2 = Sentence(sent2)
  x = embedding.embed(sentence1) # get sentence embedding of sentence 1
  se = torch.stack([token.embedding for token in sentence1])
  sent1_emb = se.numpy()
  x = embedding.embed(sentence2) # get sentence embedding of sentence 2
  se = torch.stack([token.embedding for token in sentence2])
  sent2_emb = se.numpy()
  sent1_emb = np.average(sent1_emb, axis = 0) #average sentence1 embeddings
  sent2_emb = np.average(sent2_emb, axis = 0) #average sentence2 embeddings
  #print(sent1_emb.T.shape)
  sent1_emb = negC.dot(sent1_emb.T).T  # CN sentence embeddings
  #print(sent1_emb.shape)
  sent2_emb = negC.dot(sent2_emb.T).T
  sen_emb =  negC.dot(sen_emb.T).T
  #print(sent2_emb)
  proj1 = (sent1_emb/norm(sent1_emb)).dot(sen_emb)/norm(sen_emb) #cosine similarity
  proj2 = (sent2_emb/norm(sent2_emb)).dot(sen_emb)/norm(sen_emb)
  #print(proj1,proj2)
  return proj1, proj2

In [22]:
# Get male sentences results
embedding = elmo_embedding
negC = negC_elmo
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_aa = sent_sent[(sent_sent['Race']=='African-American')] #get AA male sentences
  sent_ea = sent_sent[(sent_sent['Race']=='European')] #get EA male sentences
  sen_emb  = pick_embeddings(brown_corpus, elmo_data, str(sen)) # get certain sentiment embedding
  sen_emb = np.average(sen_emb, axis = 0)
  print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    aa.append(score1) # raw score for african american
    ea.append(score2) # raw score for european american
    aa_cn.append(score3) # CN score for african american
    ea_cn.append(score4) # CN score for european american
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(aa, ea)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


700
700
700
700


In [23]:
print('RAW vs CN data using ELMO, racial debiasing')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)


RAW vs CN data using ELMO, racial debiasing
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          -6.439617  2.227122e-10      -11.236909  4.771182e-27
fear           -4.731632  2.696043e-06       -9.460337  4.574014e-20
joy            -5.928437  4.804858e-09       -3.817702  1.466544e-04
sadness        -4.916006  1.101645e-06       -9.110037  8.550129e-19


#Bert

In [24]:
# Get male sentences results
embedding = bert_embedding
negC = negC_bert
p= []
i = 0
for sen in ['fear','sadness','joy', 'anger']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen] # get sentence of certain sentiment
  sent_aa = sent_sent[(sent_sent['Race']=='African-American')] #get AA male sentences
  sent_ea = sent_sent[(sent_sent['Race']=='European')] #get EA male sentences
  sen_emb = np.squeeze(np.asarray(senn['type_embedings'][i]))
  i+=1
  print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    
    score1, score2 = run_raw(sen_emb, sen1, sen2) #projection score on raw data
    score3, score4 = run_conceptor(sen_emb, sen1, sen2) # projection score on CN
    aa.append(score1) # raw score for african american
    ea.append(score2) # raw score for european american
    aa_cn.append(score3) # CN score for african american
    ea_cn.append(score4) # CN score for european american
  #paired t-test for raw data
  p1 = scipy.stats.ttest_rel(aa, ea)
  #paired t-test for CN data
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


700
700
700
700


In [25]:
print('RAW vs CN data using BERT, racial debiasing')
p = pd.DataFrame(p, index=['fear','sadness','joy','anger'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using BERT, racial debiasing
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
fear            5.647380  2.370618e-08       22.624530  1.853297e-85
sadness        -3.449800  5.946922e-04        3.267547  1.137887e-03
joy             4.756097  2.398224e-06        9.040590  1.512630e-18
anger           1.633738  1.027643e-01       13.350301  2.237348e-36
